In [ ]:
from main import *
import tensorflow as tf

from tqdm import tnrange, tqdm_notebook, tqdm

In [ ]:
num_classes = 2
image_shape = (160, 576)
epochs = 15
batch_size = 5
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)

# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

# Path to vgg model
vgg_path = os.path.join(data_dir, 'vgg')

In [3]:
sess = tf.Session()

In [ ]:
# def load_model(sess, model_path):
#     input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
#     nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
#     learning_rate = tf.placeholder(tf.float32)
#     correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes))
#     logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

#     saver = tf.train.Saver()
#     # Restore variables from disk.
#     saver.restore(sess, model_path)

#     g = tf.get_default_graph()
# #     print(g.get_operations())
#     logits = g.get_tensor_by_name('logits:0')
#     keep_prob = g.get_tensor_by_name('keep_prob:0')
#     input_image = g.get_tensor_by_name('image_input:0')
# #     correct_label = g.get_tensor_by_name('correct_label:0')
# #     learning_rate = g.get_tensor_by_name('learning_rate:0')
#     cross_entropy_loss = g.get_tensor_by_name('loss:0')
#     optimizer = tf.train.AdamOptimizer(learning_rate, name='Adam')
#     train_op = optimizer.minimize(cross_entropy_loss, name='train_op')

In [ ]:
# OPTIONAL: Augment Images for better results
#  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

# TODO: Build NN using load_vgg, layers, and optimize function
input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
learning_rate = tf.placeholder(tf.float32, name='learning_rate')
correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes), name='correct_label')
logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

# Initialize TF variables
sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [ ]:
weights_path = './data/ckpt/model_20epoch_0119.ckpt'
saver = tf.train.Saver() # 'Saver' misnomer! Better: Persister!

saver.restore(sess, weights_path)

In [ ]:
get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

In [ ]:
for ep in tnrange(epochs, unit=' epochs'):
    pbar = tqdm_notebook(enumerate(get_batches_fn(batch_size)), unit=' batches')
    for batch_idx, (image, label) in pbar:
        feed_dict = {input_image: image,
                     correct_label: label,
                     keep_prob: 0.50,
                     learning_rate: 1e-5,
                     }
        _, loss = sess.run([train_op, cross_entropy_loss], feed_dict)
        if batch_idx % 10 == 0:
            pbar.set_description('Epoch: %d, Batch: %d, Loss = %.3f' % (ep+1, batch_idx, loss))

In [ ]:
helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

In [ ]:
import time
saver = tf.train.Saver()

tf.add_to_collection('train_vars', input_image)
tf.add_to_collection('train_vars', correct_label)
tf.add_to_collection('train_vars', keep_prob)
tf.add_to_collection('train_vars', learning_rate)
tf.add_to_collection('train_vars', train_op)
tf.add_to_collection('train_vars', cross_entropy_loss)

tf.add_to_collection('eval_vars', input_image)
tf.add_to_collection('eval_vars', logits)
tf.add_to_collection('eval_vars', keep_prob)

tf.train.write_graph(sess.graph_def, 'data/ckpt', 'model.pb', as_text=False) # Save model
save_path = saver.save(sess, "./data/ckpt/model_15epoch_0119_good.ckpt") # Save weights

print("Model saved in file: %s" % save_path)

In [ ]:
sess.close()

In [ ]:
tf.reset_default_graph()
model_path = './data/ckpt/model.pb'
weights_path = './data/ckpt/model_20epoch_0119.ckpt'

train_elements = ['image_input:0', 'correct_label:0', 'keep_prob:0', 'learning_rate:0', 'train_op:0', 'loss:0']
eval_elements = ['image_input:0', 'logits:0', 'keep_prob:0']

with tf.Session() as sess:
    input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
    nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
    learning_rate = tf.placeholder(tf.float32)
    correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes))
    logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

#     with tf.gfile.FastGFile(model_path,'rb') as f:
#         graph_def = tf.GraphDef()
#         graph_def.ParseFromString(f.read())
#     #     input_image, correct_label, keep_prob, learning_rate, train_op, cross_entropy_loss = \
#     #                     tf.import_graph_def(graph_def, name='', return_elements=train_elements)
#         sess.graph.as_default()
#         tf.import_graph_def(graph_def, name='')#, return_elements=eval_elements)
#         sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver() # 'Saver' misnomer! Better: Persister!

    saver.restore(sess, weights_path)
    helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)    